In [109]:
import requests as rq
import json
import xmltodict as xml
import logging
import pdb
import logging.config
import yaml

In [120]:
default_path='/users/mikelee/git/TLSC/configs/logconfig.json'
default_level=logging.INFO
env_key='LOG_CFG'


with open(default_path, 'r') as f:
    config = json.load(f)
logging.config.dictConfig(config)


In [121]:
# api settings
username = 'mlee'
pw = 'Michael91'
apiget = 'https://tlsc.legalserver.org/modules/report/api_export.php?load=354&api_key=5c267c1e-c3d8-44be-a8fd-e3fcac1e021a'
# apiget = 'https://tlsc.legalserver.org//modules/report/api_export.php?load=359&api_key=5c27be11-db50-4285-abc1-dd71ac1e021a'

In [122]:
log = logging.getLogger(__name__)
 
# reconfigure the data to a nice format
# datareturn = json.dumps(xml.parse(datareturn.text))
# datareturn = json.loads(datareturn)
# datareturn = xml.parse(datareturn.text)

In [123]:
class LSClient(object):
    def __init__(self, apikey, auth, permstore, log):
        '''
        inputs:
        -------
            apikey: string
                url corresponding to the report XML API
            
            auth: tuple of strings
                (username, password) for http authentication
                
            permstore: string
                filelocation where the uploaded ids can be stored 
                persistently in case of shutdown
            
            log: logging log
                log to store runs and id updates
            
        parameters:
        -----------
            reportids: list of ints
                all the report IDs that have already been uploaded
                
            toupload: list of dicts
                row object returned by requests that are not in reportids
                (i.e. the need to be uploaded)      
        '''
        
        self.apikey = apikey
        self.auth = auth
        self.permstore = permstore
        self.log = log
    
        self.reportids = []
        self.toupload = []
    
    def _id_check(self):
        '''open the file where the permstore is and check if it is empty'''
        
        try:
            with open(self.permstore, 'r') as f:
                tmp = f.readlines()

            self.reportids = [int(line) for line in tmp]
        except OSError:
            self.log.debug('File Not Found Error. Creating file at {}'.format(self.permstore))
            
            with open(self.permstore, 'w+') as f:
                pass
 
    def _pull_report(self):
        '''
        pull the report using requesets. authentication handled using http auth
        '''
        # pull
        self.log.debug('Attempting to make request')
        datareturn = rq.get(self.apikey, auth=(self.auth[0], self.auth[1]))
        
        # check datareturn error codes
        if datareturn.status_code != 200:
            self.log.debug('Bad request. Error code: {}'.format(datareturn.status_code))
            # raise some error
            raise ValueError
            
        else:
            self.log.debug('Request made successfully')
            # reconfigure the data to a nice format
            datareturn = json.dumps(xml.parse(datareturn.text))
            datareturn = json.loads(datareturn)
            if isinstance(datareturn['report']['row'], dict):
                # only a single record was found, transfrom to list
                datareturn['report']['row'] = [datareturn['report']['row']]
            for row in datareturn['report']['row']:
                if int(row['id']) not in self.reportids:
                    # append to list of completed ids
                    self.reportids.append(int(row['id']))
                    self.toupload.append(row)
                    self.log.info('New case: {}'.format(row['id']))
            
            self.log.info(
                '\n --------------------------------------------- \n \
                Found {n} new values to be uploaded \n \
                ---------------------------------------------'.format(n=len(self.toupload)))
            
            with open(self.permstore, 'a') as f:
                for line in self.reportids:
                    f.write(str(line))
            
            self.log.debug('Permanet storage updated')
        

class TLSCRequestScrubber(object):
    def __init__(self, keymapping, errorfile, log):
        self.keyfile = keymapping
        self.log = log
        self.errorfile = errorfile
        
        self._get_mapping()
        
    def _get_mapping(self):
        try:
            with open(self.keyfile, 'r') as f:
                self.keymap = json.loads(f)
        except OSError:
            self.log.error('No key file found at: {}'.format(self.keyfile))
        
    def request_mapper(self, request):
        '''
        map the request objects onto the key values given by the map file
        
        inputs:
        -------
            request: list of dicts
                list of request objects
        '''
        
        reformatted = []
        errors = []
        
        for row in reqeuest:
            stardata = {}
            for k in row.keys():
                try:
                    stardata[self.keymap[row[k]]['starkey']] =  self.keymap[row[k]['starvalue']]
                    reformatted.append(stardata)
                except KeyError:
                    self.log.error('Key pulled from TLSC database not found. Key: {}'.format(
                        k))
                    errors.append(row)
                    
        if len(errors) != 0:            
            with open(self.errorfile, 'a') as f:
                json.dump(errors, f)
            
        self.log.info('{} matters reformatted'.format(len(reformatted)))
        self.log.info('{n} matters had errors. View them here: {e}'.format(
            n=len(errors), e=self.errorfile))
        
        return reformatted


class STARSException(Exception):
    pass

class STARSClient(object):
    '''
    Client for connecting to the STAR database and uploading scrubbed Legal Server 
    reports.
    
    inputs:
    -------
        apikey: str
            url of the STARS api 
        
        auth: tuple of strings
            (username, password) for STARS
        
        startechspec: string
            file location of the STAR tech spec YAML. Used
            to typecheck and validate input
        
        log: logging handler
    '''
    
    def __init__(self, apikey, auth, startechspec, log):
        self.apikey = apikey
        self.auth = auth
        self.startechspec = startechspec
        self.log = log
    
        self._load_tech_spec()
        
    def _typemapping(self):
        '''generate type map. removed for claritys sake '''
        self.typemap = {
            'integer': int,
            'boolean': bool,
            'array': list,
            'object': dict,
            'string': str
        }
        
    def _load_tech_spec(self):
        try:
            with open(self.startechspec, 'r') as f:
                self.startechspec = yaml.load(f)
        except OSError:
            self.log.error('No tech spec file found. Address given: {}'.format(
                self.startechspec))

        
    def _validate_input(self, lsdata):
        '''
        validate the type of each row against the STAR tech spec
        
        inputs:
        -------
            lsdata: dict
                individual row in the report to be validated
        '''

        for key, value in lsdata.itmes():   
            try:
                validate = self.startechspec['definitions']['SHIP_Beneficiary_Data']['properties'][key]['type'] 
                if validate != value:
                    self.log.error('Reformatted data has does not match TYPE in tech spec. Value: {}'.format(value))
                    raise STARSException('Reformatted data has does not match TYPE in tech spec. Value: {}'.format(value))

            except KeyError:
                self.log.error('Reformatted data has key not in tech spec. Key: {}'.format(key))
                raise STARSException('Reformatted data has key not in tech spec. Key: {}'.format(key))

    def upload_to_STARS(self, lsdata):
        '''
        do we need to upload as a list or as indivual items? 
        
        inputs:
        -------
            lsdata: list of dicts
                scrubbed Legal Server data of new cases to upload
        '''
        
        for row in lsdata:
            try:
                self._validate_input(row)
            except STARSException as e:
                self.log.error('Legal Server data is invalid', e)
            
        
    

In [124]:
lsserver = LSClient(apiget, (username, pw), '/users/mikelee/git/TLSC/testfile.txt', log)

In [125]:
lsserver._pull_report()

2018-12-29 15:48:14,778 - __main__ - INFO - New case: 351450
2018-12-29 15:48:14,780 - __main__ - INFO - 
 --------------------------------------------- 
                 Found 1 new values to be uploaded 
                 ---------------------------------------------


In [46]:
lsserver.toupload[0]['address_builtin_lookup_county_county_expn']

'Alamosa'

In [55]:
with open('./test.json', 'w') as f:
    json.dump(lsserver.toupload, f)

In [84]:
with open('./STARS_TechSpecs_031918.yaml', 'r') as f:
    x = yaml.load(f)

In [85]:
x['definitions']['SHIP_Beneficiary_Data']['properties']['beneficiaryAgeGroup']['type'] 

'integer'

In [81]:
y = int(0)

In [86]:
typemap[x['definitions']['SHIP_Beneficiary_Data']['properties']['beneficiaryAgeGroup']['type'] ] == type(y)

True

In [107]:
for k,v in lsserver.toupload[0].items():
    print(k,v)

id 351450
intake_date 2018-09-10T00:00:00-00:00
date_application None
phone_other None
identification_number 18-0351450
caseworker_name Deutsch, Melissa
age_at_intake None
lsc_compliant_assets None
email None
first Delete
person_builtin_lookup_gender_gender_expn None
last Delete
initial_gross_monthly_income None
adjusted_income_monthly 0.00
phone_home ['999-999-9999', '999-999-9999']
phone_business None
phone_mobile None
person_builtin_lookup_race_race_expn None
address_builtin_lookup_county_county_expn Alamosa
person_builtin_lookup_language_language_expn English
custom_custom_matter_how_did_beneficiary_learn_about_ship__110_custom_lookup_4ebb5d2ffcacbab3e1a44ab6139efdff_expn Not Collected
matter_builtin_lookup_problem_code_legal_problem_code_expn None
lsc_compliant_special_problem_code None
custom_custom_matter_medicaid_101_custom_lookup_23ae6ed8fbabe97de2f96e412f81607f_expn Claims/Billing
custom_custom_matter_medicare_advantage__ma___mapd__102_custom_lookup_5c990489bc8258c0c9b6bc8427

In [108]:
typemap = {
            'integer': int
            
        }